<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/05_crewAI/03_tools_customer_outreach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tools for a Customer Outreach Campaign

In this notebook, we will learn more about Tools. We'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries required are the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool

- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class.

In [ ]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, we can customize the code with our logic in the `_run` function.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using our custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

**Important Note**: In a CrewAI setup, tasks run in a strict sequence: the first task executes, its output is stored, and the next task continues using that context. Agents don't automatically talk to each other, but they collaborate through the workflow: each task assigns a specific agent, and with memory=True, all agents can access shared information accumulated from earlier steps. If delegation is enabled, an agent may hand off work to another, but otherwise each agent focuses on its assigned task while still contributing to the overall coordinated flow.

## Running the Crew

**Note**: LLMs can provide different outputs for they same input.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for my team.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_company_background.txt'.
 Tool Rea

Thought: I now know the final answer.

Final Answer: 

**DeepLearningAI Company Profile:**

DeepLearningAI is an education technology company founded by Andrew Ng, a prominent figure in the field of artificial intelligence and machine learning. The company focuses on offering online courses and resources in the AI and ML sectors, with a mission to empower the global workforce to build an AI-powered future through world-class education. DeepLearningAI provides technical training on Generative AI, Machine Learning, Deep Learning, and other related topics, reaching over 7 million learners worldwide. The company has made significant strides in the online learning platform sector and is well-positioned to continue its growth trajectory.

**Key Personnel:**

- Andrew Ng: Founder of DeepLearningAI, a recognized leader in AI and machine learning, with a background as the former Chief Scientist of Baidu and current Chairman of DeepLearningAI.
- Jonathan Heyne: Chief Operating Officer at DeepLea

Final Answer:

Subject: Elevating DeepLearningAI's Success with CrewAI's Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to congratulate you on DeepLearningAI's recent achievements in advancing AI agents' capabilities to analyze various data types and derive valuable patterns. Your commitment to empowering the global workforce through world-class education is truly commendable, and it aligns perfectly with our mission at CrewAI.

At CrewAI, we specialize in providing advanced tools for lead generation, data analysis, and personalized engagement strategies. We understand the importance of building AI-powered solutions that drive innovation and create lasting value. That's why we believe that a partnership between CrewAI and DeepLearningAI can lead to mutually beneficial outcomes, optimizing customer targeting and enhancing lead generation capabilities.

With our cutting-edge tools and strategic mindset, we can help DeepLearningAI tap into new market

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Success with CrewAI's Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to congratulate you on DeepLearningAI's recent achievements in advancing AI agents' capabilities to analyze various data types and derive valuable patterns. Your commitment to empowering the global workforce through world-class education is truly commendable, and it aligns perfectly with our mission at CrewAI.

At CrewAI, we specialize in providing advanced tools for lead generation, data analysis, and personalized engagement strategies. We understand the importance of building AI-powered solutions that drive innovation and create lasting value. That's why we believe that a partnership between CrewAI and DeepLearningAI can lead to mutually beneficial outcomes, optimizing customer targeting and enhancing lead generation capabilities.

With our cutting-edge tools and strategic mindset, we can help DeepLearningAI tap into new market segments, improve conversion rates, and establish itself as a leader in the online learning platform sector. Our personalized engagement tactics are designed to enhance customer acquisition and retention efforts, aligning perfectly with DeepLearningAI's goals and values.

I would love the opportunity to discuss how CrewAI's solutions can complement DeepLearningAI's offerings and support your continued success in the AI and ML sectors. Please let me know a convenient time for a call or meeting to explore this potential collaboration further.

Thank you for your time, Andrew. I look forward to the possibility of working together to elevate DeepLearningAI's success to new heights.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI